In [19]:
import os
from torch import optim, nn, utils, Tensor
import lightning as L
import pandas as pd
from lightning.pytorch.callbacks import EarlyStopping
from supervised_model import AlphaZeroModel,MyDataset

In [20]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
model = AlphaZeroModel()

In [22]:
# load data.csv
data = pd.read_csv('data.csv')
x = data.iloc[:, :-1].values
y = data.iloc[:, -1].values
# keep only the first digit of the labels
train_ratio = 0.85
x_train, x_test = x[:int(train_ratio*len(x))], x[int(train_ratio*len(x)):]
y_train, y_test = [int(str(i)[0]) for i in y][:int(train_ratio*len(y))], y[int(train_ratio*len(y)):]


# create a dataset
train_dataset = MyDataset(Tensor(x_train), Tensor(y_train))
test_dataset = MyDataset(Tensor(x_test), Tensor(y_test))

# create a dataloader
train_loader = utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True,num_workers=9)
test_loader = utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False,num_workers=9)


In [23]:
class TestLossCallback(L.Callback):
    def __init__(self,test_dataloader):
        self.test_dataloader = test_dataloader
    def on_train_epoch_end(self, trainer, pl_module):
        # if trainer.current_epoch % 5 != 0 :
        #     return
            
        pl_module.eval()
        correct = 0
        total = 0
        i=0
        for x, y in self.test_dataloader:
            if i>1000:
                break
            i+=1
            y = [int(a) for a in str(int(y[0]))] 
            y_hat = pl_module(x.to("mps"))
            _, predicted = y_hat.max(1)
            total += 1
            correct += 1 if (predicted in y) else 0
        print(f'Accuracy: {correct/total}')
        
        pl_module.train()
        
        

In [24]:
# add early stopping
import torch


torch.autograd.set_detect_anomaly(True)
es = EarlyStopping(monitor='train_loss',min_delta=0.01,verbose=True,mode='min',patience=5,check_on_train_epoch_end=True)
trainer = L.Trainer(min_epochs=15,max_epochs=400, limit_train_batches=128,default_root_dir="supervised",check_val_every_n_epoch=1,callbacks=[es,TestLossCallback(test_loader)])
trainer.fit(model=model, train_dataloaders=train_loader)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name        | Type       | Params
-------------------------------------------
0 | conv_layers | Sequential | 22.5 M
1 | fc_layers   | Sequential | 1.1 K 
-------------------------------------------
22.5 M    Trainable params
0         Non-trainable params
22.5 M    Total params
89.805    Total estimated model params size (MB)
/Users/alberttroussard/Documents/connect4project/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:436: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.


Training: |          | 0/? [00:00<?, ?it/s]

Metric train_loss improved. New best score: 1.622


Accuracy: 0.4595404595404595


Metric train_loss improved by 0.196 >= min_delta = 0.01. New best score: 1.426


...

Accuracy: 0.7972027972027972


Metric train_loss improved by 0.022 >= min_delta = 0.01. New best score: 0.324


Accuracy: 0.8091908091908092
Accuracy: 0.7882117882117882
Accuracy: 0.8111888111888111
Accuracy: 0.8171828171828172
Accuracy: 0.8101898101898102


Monitored metric train_loss did not improve in the last 5 records. Best score: 0.324. Signaling Trainer to stop.


Accuracy: 0.8051948051948052


In [25]:
from tqdm import tqdm

# test the model
model.eval()

correct = 0
total = 0
predictions ={0:0,1:0,2:0,3:0,4:0,5:0,6:0}
for x, y in test_loader:      
        y = [int(a) for a in str(int(y[0]))]
        y_hat = model(Tensor(x))
        _, predicted = y_hat.max(1)
        #print(f'Predicted: {(predicted)} True: {y}')
        predictions[int(predicted)] += 1
        total += 1
        correct += 1 if (predicted in y) else 0
        if total % 1000 == 0:
                print(f'Accuracy: {100 * correct / total}% at {total}/{len(test_loader)}')

print(predictions)
print(f'Accuracy: {100 * correct / total}%')



Accuracy: 80.6% at 1000/29326
Accuracy: 78.9% at 2000/29326
Accuracy: 79.0% at 3000/29326
Accuracy: 79.275% at 4000/29326
Accuracy: 79.5% at 5000/29326
Accuracy: 79.65% at 6000/29326
Accuracy: 79.65714285714286% at 7000/29326
Accuracy: 79.4375% at 8000/29326
Accuracy: 79.62222222222222% at 9000/29326
Accuracy: 79.71% at 10000/29326
Accuracy: 79.77272727272727% at 11000/29326
Accuracy: 79.65% at 12000/29326
Accuracy: 79.52307692307693% at 13000/29326
Accuracy: 79.5% at 14000/29326
Accuracy: 79.58% at 15000/29326
Accuracy: 79.5625% at 16000/29326
Accuracy: 79.69411764705882% at 17000/29326
Accuracy: 79.75555555555556% at 18000/29326
Accuracy: 79.76315789473684% at 19000/29326
Accuracy: 79.76% at 20000/29326
Accuracy: 79.81428571428572% at 21000/29326
Accuracy: 79.96363636363637% at 22000/29326
Accuracy: 79.94347826086957% at 23000/29326
Accuracy: 80.00416666666666% at 24000/29326
Accuracy: 79.884% at 25000/29326
Accuracy: 79.85384615384615% at 26000/29326
Accuracy: 79.9% at 27000/29326
A

In [26]:
# test the model
model.eval()

correct = 0
total = 0
predictions ={0:0,1:0,2:0,3:0,4:0,5:0,6:0}
train_loder_batch1 = utils.data.DataLoader(train_dataset, batch_size=1, shuffle=False)
for x, y in train_loder_batch1:    
        
        if total > 10000:
            break
        y = [y]
        y_hat = model(Tensor(x))
        #print(f"i:{y_hat}")
        _, predicted = y_hat.max(1)
        predictions[int(predicted)] += 1
        total += 1
        correct += 1 if (predicted in y) else 0
        if total % 1000 == 0:
                print(f'Accuracy: {100 * correct / total}% at {total}/{len(train_loder_batch1)}')

print(predictions)
print(f'Accuracy: {100 * correct / total}%')

Accuracy: 89.6% at 1000/166179
Accuracy: 89.35% at 2000/166179
Accuracy: 88.5% at 3000/166179
Accuracy: 88.625% at 4000/166179
Accuracy: 88.44% at 5000/166179
Accuracy: 88.66666666666667% at 6000/166179
Accuracy: 88.58571428571429% at 7000/166179
Accuracy: 88.6375% at 8000/166179
Accuracy: 88.74444444444444% at 9000/166179
Accuracy: 88.66% at 10000/166179
{0: 376, 1: 2359, 2: 2015, 3: 2651, 4: 1410, 5: 785, 6: 405}
Accuracy: 88.66113388661134%
